In [98]:
import numpy as np
from numpy import pi
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import qiskit.quantum_info as qi
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel, amplitude_damping_error
from qiskit import transpile
import matplotlib.pyplot as plt
from qiskit.providers.ibmq import least_busy
from qiskit import ClassicalRegister, QuantumRegister

In [116]:
#parameters: w1,w2=0.5   e1,e2=+/-1     cs=1   u=0.2   
# x=w1(1+e1*u/cs^2) = 0.6         equation 3 link 1
# y=w2(1+e2*u/cs^2) = 0.4         equation 3 link 2
x=0.6
y=0.4

#cxly is C1/C2 and Lambda1/Lambda2
# l1= x +/- i Sqrt(1-x^2)         in c1 it is +
# l2= y +/- i Sqrt(1-y^2)         in c2 it is -
i=1j
c1l1=np.log10(complex(x,np.sqrt(1-x**2)))
c1l2=np.log10(complex(x,-np.sqrt(1-x**2)))
c2l1=np.log10(complex(y,np.sqrt(1-y**2)))
c2l2=np.log10(complex(y,-np.sqrt(1-y**2)))

In [117]:
qc=QuantumCircuit(2)
qc.cp(c1l1,1,0, ctrl_state=0)
qc.cx(1,0,ctrl_state=0)
qc.cp(c1l1,1,0,ctrl_state=0)
qc.cx(1,0,ctrl_state=0)
qc.cp(c1l2,1,0)
qc.cx(1,0)
qc.cp(c1l2,1,0)
qc.cx(1,0)
qc.draw()

CircuitError: "Invalid param type <class 'numpy.complex128'> for gate p."

In [118]:
C11=np.exp(i*c1l1)*np.eye(4)
C12=np.exp(i*c2l1)*np.eye(4)
C21=np.exp(i*c1l2)*np.eye(4)
C22=np.exp(i*c2l2)*np.eye(4)
#print(C11,"\n\n",C12,"\n\n",C21,"\n\n",C22)

In [119]:
C1=np.round(np.block([[C11,np.zeros((4,4))],[np.zeros((4,4)),C12]]),3)
C2=np.round(np.block([[C21,np.zeros((4,4))],[np.zeros((4,4)),C22]]),3)
print(C1,"\n")
print(C2)

[[0.668+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.668+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.668+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.668+0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.604+0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.604+0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.604+0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.604+0.j]] 

[[1.496+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 1.496+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 1.496+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 1.496+0.j 0.   +0.j 0.   +0.j 